In [10]:
from data_pipeline_v4 import DataGen
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
import os
from datetime import datetime
from pathlib import Path

Set up the dataset from our generator:

In [11]:
num_epochs = 50
# Whether or not to load a model. If False/0, create a new model.
# If positive integer, load that most recent model (1=most recent, 2=second-most recent, etc.)
load_model = False

In [12]:
def _fixup_shape(x, y):
  x.set_shape([None, 259, 128]) # n, h, w, c
  y.set_shape([None]) # n, nb_classes
  return x, y

batch_size = 64
tracks = pd.read_csv('./data/processed_genres_mel.csv')

# Parse filepaths
track_fpaths = list(tracks['fpath'])
track_fpaths = ['./data/fma_medium' + fpath for fpath in track_fpaths]

# Set up generator processing function
gen = DataGen()

# Set up train and test data
data = (track_fpaths, list(tracks['parent_genre_id']))
dataset = tf.data.Dataset.from_tensor_slices(data)
dataset = dataset.map(lambda fpath, label: tuple(tf.py_function(gen.get_sample, [fpath, label], [tf.float32, tf.int32])),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE, deterministic=False)
dataset = dataset.repeat()
dataset = dataset.shuffle(buffer_size=len(track_fpaths), seed=1, reshuffle_each_iteration=False)

# Define the split ratio for train/test datasets
num_train_samples = int(0.8 * len(track_fpaths))
num_test_samples = len(track_fpaths) - num_train_samples

# Split into train and test datasets
train_dataset = dataset.take(num_train_samples)
test_dataset = dataset.skip(num_train_samples)

train_dataset = train_dataset.repeat().shuffle(buffer_size=num_train_samples,
                                               reshuffle_each_iteration=True).batch(batch_size).map(_fixup_shape).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.repeat().shuffle(buffer_size=num_test_samples,
                                             reshuffle_each_iteration=True).batch(batch_size).map(_fixup_shape).prefetch(tf.data.AUTOTUNE)

Compute the class weights for balancing:

In [13]:
genres = np.array(tracks['parent_genre_id'])
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(genres),
                                                  y=genres)

class_weights = dict(enumerate(class_weights))

class_weights

{0: 0.7109196713829302,
 1: 1.3133431703204048,
 2: 0.2196312746756909,
 3: 1.0261034255599473,
 4: 0.6935106856634016,
 5: 4.056315104166667,
 6: 0.2470068189026324,
 7: 1.5498756218905472,
 8: 10.114448051948052,
 9: 21.048986486486488,
 10: 13.20021186440678,
 11: 8.750702247191011,
 12: 2.5327235772357723,
 13: 3.2049897119341564,
 14: 1.1555081602373887,
 15: 74.17261904761905}

Add checkpoint callback for saving every few epochs:

In [14]:
# Get the current saving/loading folder
if not load_model:
    dt_now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    save_dir = f'./training/training_{dt_now}'
elif load_model > 0 and isinstance(load_model, int):
    training_dir_list = os.listdir('./training/')
    training_dir_list.sort()
    save_dir = './training/' + training_dir_list[-load_model]
else:
    raise ValueError("Incorrect model loading integer (see load_model var)")

Path(save_dir).mkdir(parents=True, exist_ok=True)

In [15]:
checkpoint_path = save_dir + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

n_batches = num_train_samples // batch_size

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 save_freq='epoch')#2*n_batches)

And one for backups so we can continue training if interrupted:

In [16]:
backup_callback = tf.keras.callbacks.BackupAndRestore(
    save_dir,
    save_freq="epoch",
    delete_checkpoint=False,
)

And one for history logging:

In [17]:
csv_logger = tf.keras.callbacks.CSVLogger(save_dir + '/history.csv', append=True)

Build and train the model:

In [18]:
if load_model:
    f = open(save_dir + '/model.json', "r")
    config = f.read()
    model = tf.keras.models.model_from_json(config)
    f.close()
else:
    model = tf.keras.models.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(300, dropout=0.2, recurrent_dropout=0.2), input_shape=(259, 128)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        # tf.keras.layers.Dense(128, activation="relu"),
        # tf.keras.layers.Dropout(0.2),
        # tf.keras.layers.Dense(64, activation="relu"),
        # tf.keras.layers.Dropout(0.2),
        # tf.keras.layers.Dense(32, activation="relu"),  
        tf.keras.layers.Dense(16)
    ])

    f = open(save_dir + '/model.json', "w")
    f.write(model.to_json())
    f.close()

model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["acc"], optimizer='adam')

history = model.fit(x=train_dataset, epochs=num_epochs,
                    validation_data=test_dataset, class_weight=class_weights,
                    steps_per_epoch=num_train_samples // batch_size,
                    validation_steps=num_test_samples // batch_size,
                    callbacks=[backup_callback, cp_callback, csv_logger])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 600)              1029600   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 600)               0         
                                                                 
 dense (Dense)               (None, 16)                9616      
                                                                 
Total params: 1,039,216
Trainable params: 1,039,216
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
311/311 [==============================] - 1488s 4s/step - loss: 2.7190 - acc: 0.1175 - val_loss: 2.6626 - val_acc: 0.0519
Epoch 2/50
311/311 [==============================] - 1338s 4s/step - loss: 2.5043 - acc: 0.2213 - val_loss: 2